In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#cosine_similarity(sentence_embedding1, sentence_embedding2)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np
#checkpoint-6042-AraBERTv02
#checkpoint-9063-MARBERTv2
checkpoint_1 = "checkpoint-9063-MARBERTv2"
tokenizer_1 = AutoTokenizer.from_pretrained(checkpoint_1, trust_remote_code=True, model_max_length=512, truncation=True)
model_1 = AutoModel.from_pretrained(checkpoint_1, trust_remote_code=True).to(device)

# Second model
#tokenizer_2 = AutoTokenizer.from_pretrained(checkpoint_2, trust_remote_code=True, model_max_length=512, truncation=True)
#model_2 = AutoModel.from_pretrained(checkpoint_2, trust_remote_code=True).to(device)

In [ ]:

data = pd.read_csv('test_set.csv', sep=',')
list_of_arrays = []

for i in range(len(data)):

  inputs_1 = tokenizer_1.encode( data['text'][i], return_tensors="pt", max_length=512, truncation=True).to(device)
  #inputs_2 = tokenizer_2.encode( data['Code'][i], return_tensors="pt", max_length=512, truncation=True).to(device)

  embedding_1 = model_1(inputs_1)
  #embedding_2 = model_2(inputs_2)
  # Convert the embedding to a Numpy array
  embedding_np_1 = embedding_1[0][0][0].cpu().detach().numpy()
  #embedding_np_2 = embedding_2[0][0][0].cpu().detach().numpy()
  #embedding_np = embedding.cpu().detach().numpy()
  #list_of_arrays.append(np.concatenate((embedding_np_1, embedding_np_2), axis=0))
  list_of_arrays.append(embedding_np_1)
  print(i)
# Ensure all arrays are of the same length
if all(len(arr) == len(list_of_arrays[0]) for arr in list_of_arrays):
    # Convert the list of arrays to a DataFrame
    df = pd.DataFrame(list_of_arrays)

In [ ]:
df[768] = data['label']
df.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
0,-0.300519,-0.600120,-0.100228,0.023889,-0.677233,0.042461,0.812080,-0.911763,0.539163,0.405911,...,0.136633,0.014386,0.168407,-0.396991,0.481234,0.177205,-0.095155,0.254624,0.142826,2
1,0.131490,-0.592935,-0.252560,-0.200858,-0.599660,0.384760,0.413056,-0.876104,0.238205,0.542747,...,0.196889,0.129850,-0.458855,-0.104531,0.070055,-0.048158,-0.794878,0.343719,0.055980,2
2,-0.485836,0.192096,0.259498,0.575999,-0.442448,-0.648520,0.453612,0.128746,0.085038,0.099233,...,0.026838,-0.270227,0.799586,-0.017558,0.684578,0.364037,0.902164,0.238576,0.345712,2
3,0.192493,-0.554371,-0.321360,-0.183048,-0.515795,0.398254,0.446575,-0.800318,0.171496,0.445282,...,0.105813,0.089609,-0.476109,-0.117893,0.199677,-0.026892,-0.906035,0.248717,0.102449,2
4,-0.549870,0.139834,0.187913,0.589530,-0.598775,-0.585258,0.375030,0.010301,0.156252,0.124876,...,0.120326,-0.256982,0.866552,0.073351,0.660863,0.484770,0.750543,0.260140,0.214191,0


Save Training Data

**Ada 02 Embeddings**

In [ ]:
!pip install openai==0.28.1

In [ ]:
!pip uninstall typing_extensions
!pip install typing_extensions

In [ ]:
# imports
!pip install openai==0.28.1
!pip uninstall typing_extensions
!pip install typing_extensions

import pandas as pd
from numpy import dot
from numpy.linalg import norm
import pickle
import openai

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

# constants
EMBEDDING_MODEL = "text-embedding-ada-002"
openai.api_key = ''

# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embedding cache
embedding_cache_path = "recommendations_embeddings_cache_ruby.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    embedding_cache=embedding_cache
) -> list:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]


embedding_from_string(data['sentence'][i])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.2 MB/s eta 0:00:00
Found existing installation: typing_extensions 4.11.0
Uninstalling typing_extensions-4.11.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/typing_extensions-4.11.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/typing_extensions.py
Proceed (Y/n)? Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_install.py", line 680, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_uninstall.py", line 375, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  Fi

KeyboardInterrupt: 

In [ ]:
data = pd.read_csv('/content/Train.csv', sep=',')
list_of_arrays = []

for i in range(len(data)):

  """inputs_1 = tokenizer_1.encode( data['sentence'][i], return_tensors="pt", max_length=512, truncation=True).to(device)
  inputs_2 = tokenizer_2.encode( data['sentence'][i], return_tensors="pt", max_length=512, truncation=True).to(device)

  embedding_1 = model_1(inputs_1)
  embedding_2 = model_2(inputs_2)
  # Convert the embedding to a Numpy array
  embedding_np_1 = embedding_1[0][0][0].cpu().detach().numpy()
  embedding_np_2 = embedding_2[0][0][0].cpu().detach().numpy()
  #embedding_np = embedding.cpu().detach().numpy()"""
  list_of_arrays.append(embedding_from_string(data['sentence'][i]))
  print(i)
# Ensure all arrays are of the same length
if all(len(arr) == len(list_of_arrays[0]) for arr in list_of_arrays):
    # Convert the list of arrays to a DataFrame
    df = pd.DataFrame(list_of_arrays)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536
0,-0.009788,0.010869,-0.004839,0.006276,-0.012681,0.025280,0.019288,0.003363,-0.032585,-0.013810,...,0.005752,-0.028509,-0.028755,-0.016484,-0.004309,0.003803,-0.000416,-0.024336,-0.018235,0
1,-0.042073,0.018731,0.007229,-0.053087,0.009969,0.012051,0.005371,0.005171,-0.031440,-0.007345,...,0.027045,-0.027222,-0.013618,-0.015435,-0.003669,-0.010491,-0.018270,-0.005880,-0.019491,0
2,-0.030342,-0.008380,-0.004577,-0.011597,-0.014418,0.000219,-0.000555,0.015144,-0.011419,-0.011289,...,0.025495,-0.039434,-0.010413,-0.023332,-0.016458,0.002632,-0.039516,-0.002251,-0.010392,0
3,-0.006279,-0.013611,0.011022,-0.041643,0.005069,0.016472,-0.005949,0.012361,-0.042730,-0.034766,...,0.009031,-0.011872,-0.019422,-0.023662,0.006099,-0.001118,-0.010411,-0.009534,-0.012782,0
4,-0.025684,0.019810,-0.001225,-0.016942,-0.006588,0.017067,-0.005652,0.008053,-0.044352,-0.025294,...,0.006407,-0.023401,-0.023457,-0.002374,0.008902,0.004615,-0.016246,-0.019615,-0.012445,0
